# Installing the NAG library and running this notebook

This notebook depends on the NAG library for Python to run. Please read the instructions in the [Readme.md](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#install) file to download, install and obtain a licence for the library.

Instruction on how to run the notebook can be found [here](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#jupyter).

# New optimization interfaces: LP

In [1]:
# imports
from naginterfaces.library import opt
from naginterfaces.base import utils
import numpy as np

### Step by step definition of the problem
We want to define the Linear Programming problem:
<center>
$$
\min c^Tx\\
l_A \leq Ax \leq u_A\\
x \geq 0
$$
</center>
Every model information is passed to an internal NAG structure via a handle through different (but simple) function calls. Array arguments are compatible with common Python data structures (tuples, lists, numpy arrays)

1. Start by initializing the handle: the problem has 5 variables

In [2]:
# Create a handle for the problem:
nvar = 5
handle = opt.handle_init(nvar)

2. Define simple bounds on the variables: $x \geq 0$

In [3]:
# define the variable bounds using numpy arrays:
bigbnd = 1.0e20
blx = np.empty(nvar)
bux = np.empty(nvar)
blx.fill(0.0)
bux.fill(bigbnd)
opt.handle_set_simplebounds(handle, bl=blx, bu=bux)

3. Define a linear objective function: $f(x) = x_1 + x_2 + x_3 + x_4 + x_5$ 

In [4]:
# define a linear objective using a python tuple 
c = (1.0, 1.0, 1.0, 1.0, 1.0)
opt.handle_set_linobj(handle, cvec=c)

4. Define the linear constraints: $-1 \leq Ax \leq 1$

In [5]:
# define 4 sparse linear constraints
nclin = 4
bla = (-1.0, -1.0, -1.0, -1.0)
bua = (1.0, 1.0, 1.0, 1.0)
nnz = 12
icola = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5]
irowa = [1, 1, 2, 1, 2, 3, 1, 2, 3, 4, 2, 4]
a = np.array([1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12.])
idlc = 0
k = opt.handle_set_linconstr(handle, bl=bla, bu=bua, irowb=irowa,
                             icolb=icola, b=a, idlc=idlc)

Optional parameter can be set to control the solver

In [6]:
# Options setting
# Set the level of output  details 
opt.handle_opt_set(handle, 'Print Level = 2')
# Deactivate the presolver to force the IPM to perform a few iterations
# (problem is solved by the presolver otherwise)
opt.handle_opt_set(handle, 'LP Presolve = No')
# tighten the convergence detection
opt.handle_opt_set(handle, 'LPIPM Stop Tolerance = 1.0e-12')

The handle is now ready to be passed to the solver

In [7]:
# Solve LP
x = np.empty(nvar)
iom = utils.FileObjManager(locus_in_output=False)
x = opt.handle_solve_lp_ipm(handle, x=x, u=None, monit=None, io_manager=iom)


 ----------------------------------------------
  E04MT, Interior point method for LP problems
 ----------------------------------------------

 Begin of Options
     Print File                    =                   9     * d
     Print Level                   =                   2     * U
     Print Options                 =                 Yes     * d
     Print Solution                =                  No     * d
     Monitoring File               =                  -1     * d
     Monitoring Level              =                   4     * d
     Lpipm Monitor Frequency       =                   0     * d

     Infinite Bound Size           =         1.00000E+20     * d
     Task                          =            Minimize     * d
     Stats Time                    =                  No     * d

     Lp Presolve                   =                  No     * U
     Lpipm Algorithm               =         Primal-dual     * d
     Lpipm Centrality Correctors   =                   

Delete cleanly the handle data when finished:

In [8]:
# destroy the handle
opt.handle_free(handle)